**Pattern 1** — Moving Average with deque

**Problem:** Compute rolling average of the last k metrics (e.g., API latency, CPU usage) from a stream.

**Task:** Given a list of numbers and window size k, return the moving average for each full window.

In [0]:
from collections import deque

def moving_average(nums, k):
    q = deque()     # holds last k values
    s = 0           # running sum of window
    out = []

    for n in nums:
        q.append(n)
        s += n

        # shrink window if too large
        if len(q) > k:
            s -= q.popleft()

        # only output when we have a full window
        if len(q) == k:
            out.append(s / k)

    return out

# Example
latencies = [100, 120, 130, 110, 90]
print(moving_average(latencies, k=3))  # [116.666..., 120.0, 110.0]

**Idea (Sliding window with deque + running sum)**

q keeps only the last k numbers.

For each new n:

Add n to q and to sum s.

If window too big, remove leftmost element and subtract from s.

Once len(q) == k, s / k is the current moving average.

**Why deque?**
deque.popleft() is O(1), while list.pop(0) is O(n). Perfect for queue-like sliding windows.

**Time & Space Complexity**

Time: Each element is appended once and popped at most once → O(n).

Space: Window holds at most k elements → O(k).

**Pattern 2** — Keep the Last N Events

**Problem:** Maintain only the last N events in memory (e.g., last 1000 user clicks or log entries) for debugging/monitoring.

**Task:** Design a structure that automatically drops the oldest event when capacity is exceeded.

In [0]:
from collections import deque

# keep last 1000 events
window = deque(maxlen=1000)

def record_event(event):
    window.append(event)   # oldest auto-dropped when len > maxlen

# Example
record_event({"user": "alice", "ts": "2025-11-23T10:01:00"})
record_event({"user": "bob", "ts": "2025-11-23T10:02:00"})

# window always has at most 1000 latest events
for e in window:
    print(e)

**Idea (Bounded queue with automatic eviction)**

deque(maxlen=N) is a fixed-capacity buffer.

When you append, and the deque is full, it automatically drops items from the left (oldest).

You never manually manage deletions.

**Why this pattern?**

Great for:

last N errors

last N user actions

last N messages for an in-memory debug buffer

You cap memory usage naturally.

**Time & Space Complexity**

Time per append: O(1) amortized (append + auto-evict).

Space: At most N elements in memory → O(N).

**Pattern 3** — Real-Time Rate Limiting Window

**Problem:** Allow at most limit requests per interval seconds per user (e.g., max 100 requests per 60 seconds).

**Task:** Given a stream of request timestamps, decide for each if it should be allowed or blocked.

In [0]:
from collections import defaultdict, deque
import time

# per-user sliding windows of timestamps
user_windows = defaultdict(lambda: deque())

def allow_request(user_id, now, limit=100, interval=60):
    """
    Return True if request is allowed, False if rate-limited.
    now is a timestamp in seconds (time.time()).
    """
    q = user_windows[user_id]

    # 1) Drop timestamps older than (now - interval)
    cutoff = now - interval
    while q and q[0] < cutoff:
        q.popleft()

    # 2) Check how many recent requests are left
    if len(q) >= limit:
        return False  # too many requests in the last interval

    # 3) Accept this request and record its timestamp
    q.append(now)
    return True

# Example
now = time.time()
print(allow_request("alice", now))        # True
print(allow_request("alice", now + 1))    # True
# after many calls within 60s, eventually it returns False


**Idea (Sliding time window per key using deque)**

For each user_id, keep a deque of recent request timestamps.

On each request:

Remove old timestamps (< now - interval) from the left.

If len(q) >= limit, block.

Else append current timestamp and allow.

**Why deque?**

Old timestamps are always at the left → cheap popleft() in O(1).

New requests go to the right via append().

**Time & Space Complexity**

Let R be the max number of requests in any sliding window for a user.

Time per request:

Each timestamp is appended once and later removed once → amortized O(1).

Space:

Per user: at most limit (or R) timestamps stored → O(R).

Overall: sum over all active users.

**deque**

Sliding windows

Last N events buffering

Rate limiting